In [1]:
from sympy import *
init_printing()

In [2]:
from sympy.utilities.iterables import partitions, multiset, multiset_permutations
import math

In [3]:
n = Symbol('n')  # sample size
mu = IndexedBase("mu")  # central moments

Suppose $X_i$ are i.i.d. with mean $\mu$ and  central moments $\mu_k = E(X-\mu)^k$.

The following function computes $E\left[\left(\sum_{i=1}^n (X_i-\mu)^2\right)^p \left(\sum_{i=1}^n (X_i-\mu)\right)^q \right]$ in terms of $\mu_k$, it will be used later to compute moments of sample variance.

In [4]:
# The follow algorithm is modified from Algorithm 1 of 
# Vegas-Sánchez-Ferrero, Gonzalo, et al. "A direct calculation of moments of the sample variance." 
# Mathematics and Computers in Simulation 82.5 (2012): 790-804
# in order to express in terms of central moments.

def ESX2pSXq(n, p, q, mu = IndexedBase("mu")):
    total = 0
    M = p + q
    for s in range(1, M + 1):
        nCs = binomial(n, s)
        parts = partitions(M, s, size = True)
        for size, part in parts:
            if size != s:
                continue
            part = [k for k, v in part.items() for j in range(v)]
            sigma = int(math.factorial(len(part)) / prod([math.factorial(x) for x in multiset(part).values()]))
            perms = multiset_permutations([i + 1 for i, v in enumerate(part) for j in range(v)])
            for perm in perms:
                c = [0]*M
                for j in range(1, M+1):
                    for i, x in enumerate(perm):
                        if x == j:
                            if i < p:
                                c[j - 1] += 2
                            else:
                                c[j - 1] += 1
                if 1 in c:
                    continue
                e = 1
                for j in range(0, M):
                    if c[j] > 0:
                        e *= mu[c[j]]
                total += nCs * sigma * e
    return total

Example 1: $E\left(\sum_{i=1}^n (X_i-\mu)^2\right) = n Var(X_i)$ 

In [5]:
ESX2pSXq(n, 1, 0)

n⋅mu[2]

Example 2: $E\left[\sum_{i=1}^n (X_i-\mu)\right]^2 = n^2 Var(\bar X)$ 

In [6]:
ESX2pSXq(n, 0, 2)

n⋅mu[2]

We are ready to compute the moments of sample variance $V_n=S_n^2$ because

$$
\begin{align*}
E(V_n^k) &= E\left[\left( \frac{1}{n-1}\sum_{i=1}^n (X_i-\bar X)^2 \right)^k\right] \\
& =E\left[\left( \frac{1}{n-1} \left( \sum_{i=1}^n  (X_i-\mu)^2 - \sum_{i=1}^n(\bar X - \mu)^2 \right) \right)^k\right] \\
&= \left( \frac{1}{n-1} \right)^k \sum_{r=0}^k \binom{n}{r} (-1)^{k-r} E\left[\left(\sum_{i=1}^n (X_i-\mu)^2\right)^r \left(\sum_{i=1}^n (\bar X -\mu)^2\right)^{k-r} \right] \\
&= \left( \frac{1}{n-1} \right)^k \sum_{r=0}^k \binom{n}{r} (-1)^{k-r} n^{r-k} E\left[\left(\sum_{i=1}^n (X_i-\mu)^2\right)^r \left(\sum_{i=1}^n (X_i-\mu)\right)^{2(k-r)} \right]
\end{align*}
$$

In [7]:
def momentV(n, k, mu = IndexedBase("mu")):
    if k == 0:
        return 1
    total = 0
    for r in range(k + 1):
        total += binomial(k, r) * n**r * (-1)**(k-r) * ESX2pSXq(n, r, 2*(k-r), mu)
    return total / n**k / (n-1)**k

In [8]:
m = [simplify(momentV(n, j)) for j in range(0, 4)]
m

⎡           2      2            2                    2           4      3      3      3
⎢          n ⋅mu[2]  - 2⋅n⋅mu[2]  + n⋅mu[4] + 3⋅mu[2]  - mu[4]  n ⋅mu[2]  - 5⋅n ⋅mu[2] 
⎢1, mu[2], ───────────────────────────────────────────────────, ───────────────────────
⎢                               n⋅(n - 1)                                              
⎣                                                                                      

      3                   2      3      2                  2      2    2               
 + 3⋅n ⋅mu[2]⋅mu[4] + 15⋅n ⋅mu[2]  - 9⋅n ⋅mu[2]⋅mu[4] - 6⋅n ⋅mu[3]  + n ⋅mu[6] - 33⋅n⋅m
───────────────────────────────────────────────────────────────────────────────────────
                                                                             2        2
                                                                            n ⋅(n - 1) 

    3                                2                       3                         
u[2]  + 21⋅n⋅mu[2]⋅mu[4] + 12⋅


Similar to want we did in the previous post, we consider Taylor’s expanding $g(x) = \sqrt{x}$.

In [9]:
x = Symbol("x")
sigma = Symbol("sigma", positive = True)
lam = IndexedBase("lambda")

In [10]:
g = sqrt(x).series(x, x0=sigma**2, n=4)
g

          3             2                                        
⎛   2    ⎞    ⎛   2    ⎞       2            ⎛          4        ⎞
⎝- σ  + x⎠    ⎝- σ  + x⎠    - σ  + x        ⎜⎛   2    ⎞        2⎟
─────────── - ─────────── + ──────── + σ + O⎝⎝- σ  + x⎠ ; x → σ ⎠
       5             3        2⋅σ                                
   16⋅σ           8⋅σ                                            

Now, we are ready to compute $E(S_n) = Eg(S_n^2)$ in terms of 
the scaled central moments $\lambda_k = \mu_k / \sigma$.

Note that $\gamma = \lambda_3$ and $\kappa = \lambda_4$ are usually called skewness and kurtosis.

In [11]:
g_coeffs = list(reversed(g.removeO().as_poly(x).all_coeffs()))
g_coeffs

⎡5⋅σ   15    -5      1  ⎤
⎢───, ────, ─────, ─────⎥
⎢ 16  16⋅σ      3      5⎥
⎣           16⋅σ   16⋅σ ⎦

In [12]:
ES = sum([g_coeffs[i] * m[i]  for i in range(0, 4)]).subs({
    mu[2]: sigma**2, 
    mu[3]: lam[3] * sigma**3, 
    mu[4]: lam[4] * sigma**4, 
    mu[6]: lam[6] * sigma**6})

In [13]:
ES.simplify()

  ⎛    4      3                 3      2          2    2              2                
σ⋅⎝16⋅n  - 2⋅n ⋅lambda[4] - 30⋅n  - 6⋅n ⋅lambda[3]  + n ⋅lambda[4] + n ⋅lambda[6] + 10⋅
───────────────────────────────────────────────────────────────────────────────────────
                                                                                       
                                                                                       

 2                 2                                                       2           
n  + 12⋅n⋅lambda[3]  + 16⋅n⋅lambda[4] - 2⋅n⋅lambda[6] - 18⋅n - 10⋅lambda[3]  - 15⋅lambd
───────────────────────────────────────────────────────────────────────────────────────
     2 ⎛ 2          ⎞                                                                  
 16⋅n ⋅⎝n  - 2⋅n + 1⎠                                                                  

                     ⎞
a[4] + lambda[6] + 30⎠
──────────────────────
                      
                      

We then expand it in terms of $1/n$

In [14]:
w = Symbol("1/n")

In [15]:
ES.subs(n, 1/w).series(w, 0, 3)

                                   ⎛               2                                  ⎞
        ⎛  σ⋅lambda[4]   σ⎞      2 ⎜  3⋅σ⋅lambda[3]    3⋅σ⋅lambda[4]   σ⋅lambda[6]   σ⎟
σ + 1/n⋅⎜- ─────────── + ─⎟ + 1/n ⋅⎜- ────────────── - ───────────── + ─────────── - ─⎟
        ⎝       8        8⎠        ⎝        8                16             16       8⎠

          
    ⎛   3⎞
 + O⎝1/n ⎠
          

Finally, we get

$$
E(S_n) =\sigma - \frac{\sigma}{8n}(\lambda_4 - 1) + \frac{\sigma}{16 n^2}(\lambda_6 - 3 \lambda_4 - 6 \lambda_3^2 - 2) + o(n^{-2})
$$

and
$$
MSE(S_n) = 2 \sigma^2 -  2 \sigma E(S_n)
$$

In [16]:
MSES = 2*sigma**2 - 2*sigma*ES
MSES.subs(n, 1/w).series(w, 0, 3)

    ⎛ 2              2⎞        ⎛   2          2      2              2              2⎞  
    ⎜σ ⋅lambda[4]   σ ⎟      2 ⎜3⋅σ ⋅lambda[3]    3⋅σ ⋅lambda[4]   σ ⋅lambda[6]   σ ⎟  
1/n⋅⎜──────────── - ──⎟ + 1/n ⋅⎜─────────────── + ────────────── - ──────────── + ──⎟ +
    ⎝     4         4 ⎠        ⎝       4                8               8         4 ⎠  

        
  ⎛   3⎞
 O⎝1/n ⎠
        

Therefore
$$
MSE(S_n) = \frac{\sigma^2}{4n}(\lambda_4 - 1) - \frac{\sigma^2}{8 n^2}(\lambda_6 - 3 \lambda_4 - 6 \lambda_3^2 - 2) + o(n^{-2}).
$$

Comparing the MSE of $\hat \sigma_n = \sqrt{\frac{\sum(X_i - \bar X)^2}{n}}$,

$$
MSE(\hat \sigma_n) = E\left[\sqrt{\frac{n-1}{n}} S_n - \sigma\right]^2 = \frac{n-1}{n} E(S_n^2) - 2 \sqrt{\frac{n-1}{n}}\sigma E(S_n) + \sigma^2.
$$

In [17]:
MSEsigmahat = (n-1)/n * sigma**2 - 2 * sigma * sqrt((n-1)/n) * ES + sigma**2

In [18]:
MSEsigmahat.subs(n, 1/w).series(w, 0, 3)

    ⎛ 2              2⎞        ⎛   2          2    2              2                2⎞  
    ⎜σ ⋅lambda[4]   σ ⎟      2 ⎜3⋅σ ⋅lambda[3]    σ ⋅lambda[4]   σ ⋅lambda[6]   5⋅σ ⎟  
1/n⋅⎜──────────── - ──⎟ + 1/n ⋅⎜─────────────── + ──────────── - ──────────── + ────⎟ +
    ⎝     4         4 ⎠        ⎝       4               4              8          8  ⎠  

        
  ⎛   3⎞
 O⎝1/n ⎠
        

which gives

$$
MSE(\hat \sigma_n) = \frac{\sigma^2}{4n}(\lambda_4 - 1) - \frac{\sigma^2}{8 n^2}(\lambda_6 - 2\lambda_4 - 6 \lambda_3^2 - 5) + o(n^{-2})
$$


In [19]:
(MSES - MSEsigmahat).subs(n, 1/w).series(w, 0, 3).coeff(w**2)

 2                2
σ ⋅lambda[4]   3⋅σ 
──────────── - ────
     8          8  